# Decorator Pattern

## Intent
Attach additional responsibilities to an object dynamically. Decorators provide a flexible alternative to subclassing for extending functionality.

## Problem
You need to add behavior to objects without:
- Creating subclasses for every combination
- Modifying existing code
- Affecting other objects of the same class

**Example**: Coffee with milk, sugar, whipped cream - combinatorial explosion!

## When to Use
✅ **Use when:**
- Add responsibilities dynamically
- Responsibilities can be withdrawn
- Extension by subclassing is impractical
- Multiple optional features

❌ **Avoid when:**
- Simple inheritance suffices
- Order of decoration matters critically
- Too many small objects created

## Note
⚠️ This is the **structural** Decorator pattern, different from Python's `@decorator` syntax!

## Example 1: Coffee Shop (Without Decorator)

**Problem**: Explosion of subclasses

In [ ]:
# WITHOUT Decorator - Class explosion!
class Coffee:
    def cost(self):
        return 2.0

class CoffeeWithMilk:
    def cost(self):
        return 2.5

class CoffeeWithSugar:
    def cost(self):
        return 2.3

class CoffeeWithMilkAndSugar:  # Every combination needs a class!
    def cost(self):
        return 2.8

# What about milk + sugar + whipped cream? Another class!
# 10 options = 1024 classes! 😱

## Implementation: Decorator Pattern

In [ ]:
from abc import ABC, abstractmethod

# Component interface
class Coffee(ABC):
    """Abstract coffee component."""
    
    @abstractmethod
    def cost(self) -> float:
        pass
    
    @abstractmethod
    def description(self) -> str:
        pass


# Concrete Component
class SimpleCoffee(Coffee):
    """Basic coffee."""
    
    def cost(self) -> float:
        return 2.0
    
    def description(self) -> str:
        return "Simple Coffee"


# Base Decorator
class CoffeeDecorator(Coffee):
    """Base decorator - wraps a Coffee object."""
    
    def __init__(self, coffee: Coffee):
        self._coffee = coffee
    
    def cost(self) -> float:
        return self._coffee.cost()
    
    def description(self) -> str:
        return self._coffee.description()


# Concrete Decorators
class MilkDecorator(CoffeeDecorator):
    """Adds milk to coffee."""
    
    def cost(self) -> float:
        return self._coffee.cost() + 0.5
    
    def description(self) -> str:
        return self._coffee.description() + ", Milk"


class SugarDecorator(CoffeeDecorator):
    """Adds sugar to coffee."""
    
    def cost(self) -> float:
        return self._coffee.cost() + 0.2
    
    def description(self) -> str:
        return self._coffee.description() + ", Sugar"


class WhippedCreamDecorator(CoffeeDecorator):
    """Adds whipped cream to coffee."""
    
    def cost(self) -> float:
        return self._coffee.cost() + 0.7
    
    def description(self) -> str:
        return self._coffee.description() + ", Whipped Cream"


# Demo
print("=== Coffee Shop Orders ===")

# Order 1: Simple coffee
coffee1 = SimpleCoffee()
print(f"{coffee1.description()}: ${coffee1.cost():.2f}")

# Order 2: Coffee with milk
coffee2 = MilkDecorator(SimpleCoffee())
print(f"{coffee2.description()}: ${coffee2.cost():.2f}")

# Order 3: Coffee with milk and sugar
coffee3 = SugarDecorator(MilkDecorator(SimpleCoffee()))
print(f"{coffee3.description()}: ${coffee3.cost():.2f}")

# Order 4: Everything!
coffee4 = WhippedCreamDecorator(
    SugarDecorator(
        MilkDecorator(SimpleCoffee())
    )
)
print(f"{coffee4.description()}: ${coffee4.cost():.2f}")

# Order 5: Double milk!
coffee5 = MilkDecorator(MilkDecorator(SimpleCoffee()))
print(f"{coffee5.description()}: ${coffee5.cost():.2f}")

## Real-World Example: Text Processing Pipeline

In [ ]:
class TextProcessor(ABC):
    """Abstract text processor."""
    
    @abstractmethod
    def process(self, text: str) -> str:
        pass


class PlainText(TextProcessor):
    """Basic text processor."""
    
    def process(self, text: str) -> str:
        return text


class TextDecorator(TextProcessor):
    """Base text decorator."""
    
    def __init__(self, processor: TextProcessor):
        self._processor = processor
    
    def process(self, text: str) -> str:
        return self._processor.process(text)


class UpperCaseDecorator(TextDecorator):
    """Converts to uppercase."""
    
    def process(self, text: str) -> str:
        return self._processor.process(text).upper()


class TrimDecorator(TextDecorator):
    """Removes whitespace."""
    
    def process(self, text: str) -> str:
        return self._processor.process(text).strip()


class HTMLDecorator(TextDecorator):
    """Wraps in HTML tags."""
    
    def __init__(self, processor: TextProcessor, tag: str = "p"):
        super().__init__(processor)
        self.tag = tag
    
    def process(self, text: str) -> str:
        processed = self._processor.process(text)
        return f"<{self.tag}>{processed}</{self.tag}>"


class MarkdownDecorator(TextDecorator):
    """Adds markdown formatting."""
    
    def __init__(self, processor: TextProcessor, style: str = "bold"):
        super().__init__(processor)
        self.style = style
    
    def process(self, text: str) -> str:
        processed = self._processor.process(text)
        if self.style == "bold":
            return f"**{processed}**"
        elif self.style == "italic":
            return f"*{processed}*"
        return processed


# Demo
print("=== Text Processing Pipeline ===")

text = "  hello world  "

# Pipeline 1: Trim and uppercase
processor1 = UpperCaseDecorator(TrimDecorator(PlainText()))
print(f"Trimmed + Upper: '{processor1.process(text)}'")

# Pipeline 2: Trim, uppercase, and wrap in HTML
processor2 = HTMLDecorator(
    UpperCaseDecorator(
        TrimDecorator(PlainText())
    ),
    tag="h1"
)
print(f"Full HTML: {processor2.process(text)}")

# Pipeline 3: Markdown bold
processor3 = MarkdownDecorator(
    TrimDecorator(PlainText()),
    style="bold"
)
print(f"Markdown: {processor3.process(text)}")

## Real-World Example: Data Stream with Encryption & Compression

In [ ]:
import zlib
import base64

class DataSource(ABC):
    """Abstract data source."""
    
    @abstractmethod
    def write(self, data: str) -> None:
        pass
    
    @abstractmethod
    def read(self) -> str:
        pass


class FileDataSource(DataSource):
    """Basic file data source."""
    
    def __init__(self):
        self.data = ""
    
    def write(self, data: str) -> None:
        self.data = data
        print(f"📁 Writing to file: {len(data)} bytes")
    
    def read(self) -> str:
        print(f"📁 Reading from file: {len(self.data)} bytes")
        return self.data


class DataSourceDecorator(DataSource):
    """Base decorator for data sources."""
    
    def __init__(self, source: DataSource):
        self._source = source
    
    def write(self, data: str) -> None:
        self._source.write(data)
    
    def read(self) -> str:
        return self._source.read()


class CompressionDecorator(DataSourceDecorator):
    """Adds compression."""
    
    def write(self, data: str) -> None:
        compressed = zlib.compress(data.encode())
        print(f"🗜️  Compressing: {len(data)} → {len(compressed)} bytes")
        self._source.write(base64.b64encode(compressed).decode())
    
    def read(self) -> str:
        data = self._source.read()
        decompressed = zlib.decompress(base64.b64decode(data))
        print(f"🗜️  Decompressing: {len(data)} → {len(decompressed)} bytes")
        return decompressed.decode()


class EncryptionDecorator(DataSourceDecorator):
    """Adds encryption (simple XOR for demo)."""
    
    def __init__(self, source: DataSource, key: int = 42):
        super().__init__(source)
        self.key = key
    
    def write(self, data: str) -> None:
        encrypted = self._xor_encrypt(data)
        print(f"🔐 Encrypting data")
        self._source.write(encrypted)
    
    def read(self) -> str:
        data = self._source.read()
        print(f"🔐 Decrypting data")
        return self._xor_encrypt(data)  # XOR is symmetric
    
    def _xor_encrypt(self, data: str) -> str:
        return ''.join(chr(ord(c) ^ self.key) for c in data)


# Demo
print("=== Data Stream Processing ===")

data = "Secret message! " * 20
print(f"\nOriginal data: {len(data)} bytes\n")

# Just file storage
print("--- Simple Storage ---")
source1 = FileDataSource()
source1.write(data)
read_data = source1.read()
print(f"Verified: {read_data == data}\n")

# With compression
print("--- With Compression ---")
source2 = CompressionDecorator(FileDataSource())
source2.write(data)
read_data = source2.read()
print(f"Verified: {read_data == data}\n")

# With compression and encryption
print("--- With Compression + Encryption ---")
source3 = EncryptionDecorator(
    CompressionDecorator(FileDataSource())
)
source3.write(data)
read_data = source3.read()
print(f"Verified: {read_data == data}")

## Python's @decorator vs Decorator Pattern

**Different concepts!**

### Python @decorator (Function decorator)
```python
@log
def my_function():
    pass
```

### Decorator Pattern (Object decorator)
```python
coffee = MilkDecorator(SimpleCoffee())
```

## Advantages & Disadvantages

### ✅ Advantages
1. **More flexible than inheritance**: Add/remove at runtime
2. **Single Responsibility**: Each decorator has one job
3. **Open/Closed Principle**: Extend without modifying
4. **Avoid class explosion**: No need for every combination
5. **Composable**: Mix and match decorators

### ❌ Disadvantages
1. **Many small objects**: Can be hard to debug
2. **Order matters**: Different order = different result
3. **Type checking issues**: Decorators change type
4. **Complexity**: More objects to manage

## Common Pitfalls

### 1. Breaking Interface
```python
# Bad - decorator doesn't implement full interface
class BadDecorator(CoffeeDecorator):
    def cost(self):
        return self._coffee.cost() + 1
    # Missing description()!
```

### 2. Decorator Order Issues
```python
# Order matters!
# Encrypt then Compress vs Compress then Encrypt
# Usually: Compress → Encrypt (better compression)
```

## Related Patterns

- **Adapter**: Changes interface, Decorator adds responsibility
- **Composite**: Aggregates objects, Decorator adds behavior
- **Strategy**: Changes algorithm, Decorator adds behavior
- **Proxy**: Controls access, Decorator adds behavior

## Best Practices

1. **Keep interface consistent**: All decorators should implement full interface
2. **Document order**: When order matters, document it
3. **Consider composition**: Sometimes simple composition is better
4. **Limit depth**: Too many decorators = hard to understand
5. **Use base decorator**: Provides default pass-through behavior

## Summary

Decorator pattern enables:
- Dynamic behavior addition
- Flexible alternatives to subclassing
- Composable functionality
- Runtime responsibility assignment

Perfect for: Middleware, I/O streams, UI components, text formatting, logging layers.

**Key Insight**: Wrap objects to add behavior, maintaining the same interface!